In [70]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import re
import datetime
import time
from urllib import parse
path = r"D:\chromedriver.exe"

In [75]:
def html_parser(url, dynamic=True, delay=0, headless=False):
    if not dynamic:
        return BeautifulSoup(requests.get(url).text, "html.parser")
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('headless')
    try:
        driver = webdriver.Chrome(executable_path=r"D:\chromedriver.exe", options=options)
        driver.get(url)
        time.sleep(delay)
        source = driver.page_source
        driver.quit()
        return BeautifulSoup(source, "html.parser")
    except:
        print("html parser got an error")
        return None

def get_text(target):
    if hasattr(target, "text"):
        return target.text
    try:
        return list(map(get_text, target))
    except:
        return ""

In [74]:
my_url = r"https://namu.wiki/w/%EC%82%AC%EC%9E%90%EC%9B%90%EC%88%AD%EC%9D%B4"
print(html_parser(my_url))

<html class="no-js" lang="en-US"><!--<![endif]--><head>
<title>Please Wait... | Cloudflare</title>
<meta id="captcha-bypass" name="captcha-bypass"/>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" media="screen,projection" rel="stylesheet" type="text/css"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" type="text/css" media="screen,projection" /><![endif]-->
<style type="text/css">body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function () {
      var cookieEl = document.getElementById('cookie-alert');
      cookieEl.style.display = 'block';
    }

In [32]:
def geturl(parsed_html:BeautifulSoup):
    result = []
    elements = parsed_html.select("tbody > tr > td:nth-child(1) > a:nth-child(1)")
    for element in elements:
        result.append("https://namu.wiki" + element.attrs.get("href"))
    return result

def gettime(parsed_html:BeautifulSoup):
    alltimes = parsed_html.select("tr time")
    lasttime = datetime.datetime(*map(int, re.split("[-: ]+", alltimes[0].text)))
    firsttime = datetime.datetime(*map(int, re.split("[-: ]+", alltimes[-1].text)))
    return (lasttime - firsttime).seconds//2
    

In [19]:
page_urls = {}
origin_url = r"https://namu.wiki/RecentChanges"
url_limit = int(input())
while True:
    namu_recent = html_parser(origin_url, 5)
    if namu_recent is not None:
        for url in html_parser(namu_recent):
            page_urls[url] = page_urls.get(url, 0) + 1
        print("url number:", len(page_urls))
        if len(page_urls) > url_limit:
            break
        timedif = gettime(namu_recent)
        print(f"waiting time: {timedif}s")
        time.sleep(timedif)

url number: 81


In [66]:
print(page_urls)

{'https://namu.wiki/w/%EC%9D%B4%EC%A0%95%EC%88%98(%EB%B2%95%EC%A1%B0%EC%9D%B8)': 1, 'https://namu.wiki/w/%EC%A0%9C%EC%8A%A4%ED%84%B0%20%EC%BD%A9%ED%94%BC': 2, 'https://namu.wiki/w/LESSERAFIM': 3, 'https://namu.wiki/w/%EC%9E%90%ED%87%B4': 2, 'https://namu.wiki/w/%ED%86%A0%EB%8B%88%ED%86%A0%EB%8B%88%20%EC%B5%B8%ED%8C%8C': 1, 'https://namu.wiki/w/%ED%8C%8C%EC%9D%BC:attachment/%EC%84%B1%EB%A3%A1/%EC%8B%A0%EC%A0%95%EB%AC%B4%EB%AC%B8%EC%84%B1%EB%A3%A1.jpg': 1, 'https://namu.wiki/w/%ED%95%B4%EB%A7%88%EB%A3%A8%EA%B3%B5%EC%9B%90%EB%A1%9C': 1, 'https://namu.wiki/w/%EB%84%A4%EB%AA%A8%EB%B0%94%EC%A7%80%20%EC%8A%A4%ED%8F%B0%EC%A7%80%EB%B0%A5': 1, 'https://namu.wiki/w/2038%EB%85%84%20%EB%AC%B8%EC%A0%9C': 1, 'https://namu.wiki/w/%EC%9B%94%EB%93%9C(%EA%B8%B0%EA%B3%84%EC%A0%84%EB%8C%80%20%EC%A0%A0%EC%B9%B4%EC%9D%B4%EC%A0%80)/%EB%AA%A9%EB%A1%9D': 1, 'https://namu.wiki/w/%EC%9D%B8%EC%B2%9C%EB%8C%80%ED%95%99%EA%B5%90/%EC%B6%9C%EC%8B%A0%20%EC%9D%B8%EB%AC%BC': 5, 'https://namu.wiki/w/%EC%97%B0%EC%84%B8%EB%8

In [104]:
cnt = 0
titles = []
categories = []
paragraphs = []
for target_url in page_urls:
    if cnt >= 2:
        break
    cnt += 1
    namu_keyword = html_parser(target_url)
    if namu_keyword is not None:
        article = namu_keyword.select_one("article")
        title = get_text(article.select_one("h1"))
        raw_categories = list(article.select_one("h2").next_siblings)[-1].select("li")
        category = "&".join(get_text(raw_categories))
        for i in article.select("table"):
            i.clear()
        if article.select_one(".wiki-heading-content") is None:
            raw_paragraphs = article.select(".wiki-paragraph")
        else:
            raw_paragraphs = article.select(".wiki-heading-content>.wiki-paragraph")
        paragraph = "&".join(get_text(raw_paragraphs))
        titles.append(title)
        categories.append(category)
        paragraphs.append(paragraph)

In [105]:
import pandas
df = pandas.DataFrame(data={"title":titles, "category":categories, "paragraph":paragraphs})
df

,title,category,paragraph
0,이정수(법조인),대한민국의 법조인&문재인 정부/인사&서울특별시 출신 인물&1969년 출생&남강고등학...,대한민국의 검사. 제62대 서울중앙지방검찰청 검사장이다.&1969년 서울특별시에서 ...
1,제스터 콩피,유희왕/OCG/마법사족,유희왕의 하급 효과 몬스터 카드.&원작에서는 예거가 사용.아무 조건이나 제약 없이 ...


In [65]:
wiki_url = r"https://ko.wikipedia.org/wiki/%ED%8A%B9%EC%88%98:%EC%B5%9C%EA%B7%BC%EB%B0%94%EB%80%9C?hidebots=1&hidecategorization=1&hideWikibase=1&limit=50&days=7&urlversion=2"
wiki_recent = html_parser(wiki_url, dynamic=False)
if wiki_recent is not None:
    article = wiki_recent.select_one(".special")
    titles = article.select(".mw-title>a")
for title in titles:
    target_title = get_text(title)
    target_url = "https://ko.wikipedia.org" + title.attrs["href"]
    wiki_keyword = html_parser(target_url, dynamic=False)
    target_description = get_text(wiki_keyword.select_one(".mw-parser-output > p"))
    print("title:", target_title)
    print("url:", target_url)
    print("description:", target_description)

<class 'bs4.element.Tag'>
title: 한국케이블TV푸른방송
url: https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B5%AD%EC%BC%80%EC%9D%B4%EB%B8%94TV%ED%91%B8%EB%A5%B8%EB%B0%A9%EC%86%A1
description: 한국케이블TV푸른방송 주식회사(약칭 GCS)는 대구광역시 달서구, 달성군을 서비스 구역으로 하는 대한민국의 케이블SO사업자이다.[1] 1993년 9월 29일 홍진종합유선방송으로서 설립되어 오늘날에 이르고 있다.

title: 아이파크
url: https://ko.wikipedia.org/wiki/%EC%95%84%EC%9D%B4%ED%8C%8C%ED%81%AC
description: 아이파크는 HDC현대산업개발이 2001년 3월에 런칭한 아파트, 주상복합, 오피스텔 등 주거 통합 브랜드다. IPARK는 '시간이 지날수록 더 큰 행복을 주고 삶을 아릅답게 만드는 공간'으로 '더 나은 삶을 꿈꾸게하는 혁신', '시간이 지날수록 더욱더 높아지는 가치', '미학적인 만족과 경험', '견고한 신뢰'를 지향한다.
[1]
[2]
현대아파트, 아이파크 등 HDC현대산업개발이 전국에 공급한 주택은 40여년간 41만 가구에 달한다.

title: 유미의 세포들 2
url: https://ko.wikipedia.org/w/index.php?title=%EC%9C%A0%EB%AF%B8%EC%9D%98_%EC%84%B8%ED%8F%AC%EB%93%A4_2&redirect=no
description: 《유미의 세포들 2》은 2022년부터 TVING과 tvN에서 동시 방영될  TVING 오리지널 드라마이다. 동명의 네이버 웹툰을 원작으로 한다.

title: 이라클리오스
url: https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%9D%BC%ED%81%B4%EB%A6%AC%EC%98%A4%EC%8A%A4
description: 이라클리오스 또는 헤라클

KeyboardInterrupt: 